In [1]:
import numpy as np
import tensorflow as tf

/Users/OG/Desktop/Study/jupyter/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/OG/Desktop/Study/jupyter/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
batch_size = 128
embedding_dimension = 64
# embedding_dimension = 96
num_classes = 2
hidden_layer_size = 32
times_steps = 6
element_size = 1

In [3]:
digit_to_word_map = {1: "One", 2: "Two", 3: "Three", 4: "Four", 5: "Five",
                     6: "Six", 7: "Seven", 8: "Eight", 9: "Nine"}
digit_to_word_map[0] = "PAD"

even_sentences = []
odd_sentences = []
seqlens = []

# 입력 데이터 정의

In [4]:
for i in range(10000):
    rand_seq_len = np.random.choice(range(3, 7))
    seqlens.append(rand_seq_len)
    rand_odd_ints = np.random.choice(range(1, 10, 2),
                                     rand_seq_len)
    rand_even_ints = np.random.choice(range(2, 10, 2),
                                      rand_seq_len)

    if rand_seq_len < 6:
        rand_odd_ints = np.append(rand_odd_ints,
                                  [0]*(6-rand_seq_len))
        rand_even_ints = np.append(rand_even_ints,
                                   [0]*(6-rand_seq_len))

    even_sentences.append(" ".join([digit_to_word_map[r] for r in rand_odd_ints]))
    odd_sentences.append(" ".join([digit_to_word_map[r] for r in rand_even_ints]))


In [5]:
even_sentences[0:5]

['Seven One Five Five Seven PAD',
 'One Seven One Five Seven PAD',
 'Five Nine Nine Five One One',
 'Seven Five Nine PAD PAD PAD',
 'One Five Nine Three Seven PAD']

In [6]:
odd_sentences[0:10]

['Six Eight Four Eight Two PAD',
 'Six Eight Eight Eight Eight PAD',
 'Four Six Two Eight Eight Four',
 'Four Four Two PAD PAD PAD',
 'Six Eight Eight Eight Six PAD',
 'Four Eight Eight Two PAD PAD',
 'Four Six Eight Eight PAD PAD',
 'Eight Six Four Six PAD PAD',
 'Six Six Four Six PAD PAD',
 'Two Four Four PAD PAD PAD']

In [7]:
data = even_sentences+odd_sentences
seqlens *= 2
labels = [1] * 10000 + [0] * 10000
for i in range(len(labels)):
    label = labels[i]
    one_hot_encoding = [0]*2
    one_hot_encoding[label] = 1
    labels[i] = one_hot_encoding


In [8]:
word2index_map = {}
index = 0
for sent in data:
    for word in sent.lower().split():
        if word not in word2index_map:
            word2index_map[word] = index
            index += 1

In [9]:
index2word_map = {index: word for word, index in word2index_map.items()}

vocabulary_size = len(index2word_map)

data_indices = list(range(len(data)))
np.random.shuffle(data_indices)
data = np.array(data)[data_indices]
labels = np.array(labels)[data_indices]
seqlens = np.array(seqlens)[data_indices]
train_x = data[:10000]
train_y = labels[:10000]
train_seqlens = seqlens[:10000]

test_x = data[10000:]
test_y = labels[10000:]
test_seqlens = seqlens[10000:]


# 모델 학습 전 함수 정의

In [10]:
def get_sentence_batch(batch_size, data_x,
                       data_y, data_seqlens):
    instance_indices = list(range(len(data_x)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    x = [[word2index_map[word] for word in data_x[i].lower().split()]
         for i in batch]
    y = [data_y[i] for i in batch]
    seqlens = [data_seqlens[i] for i in batch]
    return x, y, seqlens

In [11]:
_inputs = tf.placeholder(tf.int32, shape=[batch_size, times_steps])
_labels = tf.placeholder(tf.float32, shape=[batch_size, num_classes])
# seqlens for dynamic calculation
_seqlens = tf.placeholder(tf.int32, shape=[batch_size])

In [12]:
with tf.name_scope("embeddings"):
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size,
                           embedding_dimension],
                          -1.0, 1.0), name='embedding')
    embed = tf.nn.embedding_lookup(embeddings, _inputs)
    # https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup
    # https://stackoverflow.com/questions/34870614/what-does-tf-nn-embedding-lookup-function-do

In [13]:
embed

<tf.Tensor 'embeddings/embedding_lookup:0' shape=(128, 6, 64) dtype=float32>

In [14]:
_seqlens

<tf.Tensor 'Placeholder_2:0' shape=(128,) dtype=int32>

### 단일 lstm 용 함수

In [15]:
# with tf.variable_scope("lstm"):
#     lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_layer_size,
#                                              forget_bias=1.0)
#     outputs, states = tf.nn.dynamic_rnn(lstm_cell, embed,
#                                         sequence_length=_seqlens,
#                                         dtype=tf.float32)


### multiple lstm 용 함수

In [16]:
batch_size

128

In [17]:
num_LSTM_layers = 2
# 64 -> 32 -> 32
# why is this not working??
with tf.variable_scope("lstm"):
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_layer_size,
                                             forget_bias=1.0)
    cell = tf.contrib.rnn.MultiRNNCell(cells=[lstm_cell]*num_LSTM_layers,
                                       state_is_tuple=True)
    outputs, states = tf.nn.dynamic_rnn(cell, embed,
                                        sequence_length = _seqlens, dtype=tf.float32)


ValueError: Dimensions must be equal, but are 64 and 96 for 'lstm/rnn/while/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/MatMul_1' (op: 'MatMul') with input shapes: [128,64], [96,128].

In [ ]:
weights = {
    'linear_layer': tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes],
                                                    mean=0, stddev=.01))
}
biases = {
    'linear_layer': tf.Variable(tf.truncated_normal([num_classes], mean=0, stddev=.01))
}

In [ ]:
### 단일 lstm
# final_output = tf.matmul(states[1],
#                         weights["linear_layer"]) + biases["linear_layer"]

### 다중 lstm
#extract the final state and use in a linear layer
final_output = tf.matmul(states[num_LSTM_layers-1][1],
                         weights["linear_layer"]) + biases["linear_layer"]


In [ ]:


softmax = tf.nn.softmax_cross_entropy_with_logits(logits=final_output,
                                                  labels=_labels)
cross_entropy = tf.reduce_mean(softmax)

train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(_labels, 1),
                              tf.argmax(final_output, 1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction,
                                   tf.float32)))*100


# 모델 학습

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(1000):
        x_batch, y_batch, seqlen_batch = get_sentence_batch(batch_size,
                                                            train_x, train_y,
                                                            train_seqlens)
        sess.run(train_step, feed_dict={_inputs: x_batch, _labels: y_batch,
                                        _seqlens: seqlen_batch})

        if step % 100 == 0:
            acc = sess.run(accuracy, feed_dict={_inputs: x_batch,
                                                _labels: y_batch,
                                                _seqlens: seqlen_batch})
            print("Accuracy at %d: %.5f" % (step, acc))

    for test_batch in range(5):
        x_test, y_test, seqlen_test = get_sentence_batch(batch_size,
                                                         test_x, test_y,
                                                         test_seqlens)
        batch_pred, batch_acc = sess.run([tf.argmax(final_output, 1), accuracy],
                                         feed_dict={_inputs: x_test,
                                                    _labels: y_test,
                                                    _seqlens: seqlen_test})
        print("Test batch accuracy %d: %.5f" % (test_batch, batch_acc))

    output_example = sess.run([outputs], feed_dict={_inputs: x_test,
                                                    _labels: y_test,
                                                    _seqlens: seqlen_test})
    states_example = sess.run([states[1]], feed_dict={_inputs: x_test,
                                                      _labels: y_test,
                                                      _seqlens: seqlen_test})

NameError: name 'y_test' is not defined